In [1]:
import pandas as pd
import json
from opinion_overlap_eval import OpinionEval

In [24]:
def unmatched_elecat(stx):
    category_subs = {'general': 'overall', 
                 'style_options': "styles and options",
                 'design_features': 'features',
                 'operation_performance': 'performance'}
    reverse_sub = {v :k for k,v in category_subs.items()}
    
    sansthecat = stx.split(' ')[1:]
    # print(stx, sansthecat)
    if sansthecat[0] in ['and', 'or', 'the']:
        sansthecat = sansthecat[1:]

    ent, cat = None, None
    
    if len(sansthecat) <= 1:
        ent = ''.join(sansthecat)
        cat = 'general'
    else:
        ent = sansthecat[0]
        cat = ' '.join(sansthecat[1:])
        if cat in reverse_sub.keys():
            cat = reverse_sub[cat]
            # print(ent, cat)
    return [ent, cat]

def entity_category_mvp(stx):
    entcat = stx.split(' ')
    if len(entcat) != 2:
        category = 'general'
        entity = entcat[0]
    else:
        entity = entcat[0]
        category = entcat[1]
    return [entity, category]

In [2]:
# colname = ['id', 'target_entity', 'aspect_category','aspect_term', 'sentiment_polarity', 'sentiment_expression']

colname = ['id', 'aspect_term', 'sentiment_polarity', 'sentiment_expression']

In [31]:
RES = "MVP" #GEN_SCL_NAT
result_list = []
colname = ['target_entity', 'aspect_category','aspect_term', 'sentiment_polarity', 'sentiment_expression']
mapping_funct = entity_category_mvp if RES == "MVP" else unmatched_elecat
with open('../ontology_experiments/results/final_result/acos-comparison/ours24_mvp.json') as f:
    pred_res = json.load(f)
    for rec in pred_res:
        pred1 = [{colname[i]:v for i, v in enumerate(mapping_funct(val[0])+val[1:])} for val in rec['pred']]
        gold1 = [{colname[i]:v for i, v in enumerate(mapping_funct(val[0])+val[1:])} for val in rec['gold']]
        # # pred1 = [{colname[i]:v for i, v in enumerate(val[1:])} for val in rec['pred']]
        # # gold1 = [{colname[i]:v for i, v in enumerate(val[1:])} for val in rec['gold']]
        result_list.append({
            'gold': gold1,
            'pred': pred1
        })
        

In [32]:
result_list[1]

{'gold': [{'target_entity': 'book',
   'aspect_category': 'general',
   'aspect_term': 'book',
   'sentiment_polarity': 'great',
   'sentiment_expression': 'really like'},
  {'target_entity': 'content',
   'aspect_category': 'plot',
   'aspect_term': 'ending',
   'sentiment_polarity': 'bad',
   'sentiment_expression': 'not be particularly satisfied'}],
 'pred': [{'target_entity': 'restaurant',
   'aspect_category': 'general',
   'aspect_term': 'null',
   'sentiment_polarity': 'great',
   'sentiment_expression': 'like'},
  {'target_entity': 'book',
   'aspect_category': 'design_features',
   'aspect_term': 'book',
   'sentiment_polarity': 'bad',
   'sentiment_expression': 'satisfied'}]}

In [33]:
TP = 0
TOT_PRED = 0
TOT_GOLD = 0
opeval = OpinionEval(colname)
for i in range(len(result_list)):
    gold_arx = result_list[i]['gold']
    pred_arx = result_list[i]['pred']
    tp, _, _ = opeval.one_input_pred_metrics(gold_arx, pred_arx)
    TP += tp
    TOT_PRED += len(pred_arx)
    TOT_GOLD += len(gold_arx)

precision = TP/TOT_PRED
recall = TP/TOT_GOLD
f1_score = 2 * precision * recall / (precision + recall)
 
print(f'precision: {precision}, recall: {recall}, f1-score: {f1_score}')

precision: 0.660472972972973, recall: 0.7294776119402985, f1-score: 0.6932624113475179


In [34]:
import os
'../on'

'../on'

In [3]:
opeval = OpinionEval(colname)
res_df = pd.read_csv('../ontology_experiments/results/final_result/gpt-4o/nlprompt/csv/eg_desc_format.csv')
gold_df = pd.read_csv('../ontology_experiments/results/final_result/gold_annotations_fin_3.csv')
replace_dict = {'sentiment_polarity': {'POS':'positive', 'NEG':'negative', 'NEU':'neutral'}}
gold_df = gold_df.replace(replace_dict).fillna('NULL')
res_df = res_df.fillna('NULL')

In [4]:
gdict = opeval.get_all_gold_rows(gold_df, 'b-2')
pdict = opeval.get_predicted_rows(res_df, 'b-2')

In [5]:
print(gdict)
print(pdict)

[{'id': 'b-2', 'aspect_term': 'book', 'sentiment_polarity': 'positive', 'sentiment_expression': 'really like'}]
[{'id': 'b-2', 'aspect_term': 'it', 'sentiment_polarity': 'positive', 'sentiment_expression': 'really like'}]


In [6]:
opeval.eval_opinions(gold_df, res_df)

{'precision': 0.7524509803921566,
 'recall': 0.7415458937198065,
 'f1': 0.746958637469586}